In [25]:
%env TF_CPP_MIN_LOG_LEVEL=3

import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import datetime
from pathlib import Path

import plotly.express as px

import tensorflow as tf

from importlib import reload
import keras_model_creator
import sentinel_utils
import plot_utils
from data_generator import DataGenerator

env: TF_CPP_MIN_LOG_LEVEL=3


In [43]:
reload(sentinel_utils)

model_dir = Path('models', 'selected_model')
shards_dir = Path.home().joinpath('sentinel_data', 'shards')

utils = sentinel_utils.SentinelUtils(min_occurrences=20000)

selected_classes = utils.get_processed_labels()

data_summary = utils.get_data_summary(
    shards_dir, selected_classes
)

loss = 'binary_crossentropy'
batch_size = 64
base_filters = 32

model_parent_dir = Path('models')
model_dir = model_parent_dir.joinpath(
    f'{loss}-{len(selected_classes.index)}'
    f'-{selected_classes.shape[1]}-{batch_size}-{base_filters}'
)
model_dir.mkdir(parents=True, exist_ok=True)

params = dict(
    loss=loss,
    base_filters=base_filters,
    shards_dir=shards_dir,
    selected_classes=selected_classes,
    data_summary=data_summary,
    batch_size=64,
    dropout=0.2,
    model_dir=model_dir,
    epochs=10,
    overwrite=False,
    verbose=1,
    print_log=0
)

In [45]:
reload(keras_model_creator)
keras_model_creator.KerasModelCreator(**params).run()

TypeError: tuple expected at most 1 argument, got 2